# SEFA

## Test Capture

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'NONE'

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/simpleserial-sefa
make PLATFORM=$1 CRYPTO_TARGET=$2

In [ ]:
%run "Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = '../hardware/victims/firmware/simpleserial-sefa/simpleserial-sefa-{}.hex'.format(PLATFORM)
cw.program_target(scope, prog, fw_path)

In [ ]:
scope.clock.adc_src = "clkgen_x1"
scope.io.hs2 = "clkgen"
scope.adc.timeout = 0.1

ktp = cw.ktp.Basic()
key, text = ktp.next()

trace = cw.capture_trace(scope, target, text, key)

ciphertext = trace.textout
sbox_in = text
sbox_out = ciphertext

# input
a = (sbox_in[0] ^ sbox_in[5]) & 0xFF
b = (sbox_in[1] ^ sbox_in[6]) & 0xFF
c = (sbox_in[2] ^ sbox_in[7]) & 0xFF
d = (sbox_in[3] ^ sbox_in[8]) & 0xFF
e = (sbox_in[4] ^ sbox_in[9]) & 0xFF

# observed output
aa = (sbox_out[0] ^ sbox_out[5]) & 0xFF
bb = (sbox_out[1] ^ sbox_out[6]) & 0xFF
cc = (sbox_out[2] ^ sbox_out[7]) & 0xFF
dd = (sbox_out[3] ^ sbox_out[8]) & 0xFF
ee = (sbox_out[4] ^ sbox_out[9]) & 0xFF

# redundant computation
aa_cor = a ^ ((b^0xFF) & c);
bb_cor = b ^ ((c^0xFF) & d);
cc_cor = c ^ ((d^0xFF) & e);
dd_cor = d ^ ((e^0xFF) & a);
ee_cor = e ^ ((a^0xFF) & b);

print(aa)
print(aa_cor)
print(bb)
print(bb_cor)
print(cc)
print(cc_cor)
print(dd)
print(dd_cor)
print(ee)
print(ee_cor)

print('')
print(scope.adc.trig_count)

In [ ]:
#Capture Traces
import numpy as np
import time
import chipwhisperer.common.results.glitch as glitch

def reboot_flush():            
    scope.io.pdic = False
    time.sleep(0.1)
    scope.io.pdic = "high_z"
    time.sleep(0.1)
    target.flush()

ktp = cw.ktp.Basic()
key, text = ktp.next()

SBOX = [
    0, 9, 18, 11, 5, 12, 22, 15,
    10, 3, 24, 1, 13, 4, 30, 7,
    20, 21, 6, 23, 17, 16, 2, 19,
    26, 27, 8, 25, 29, 28, 14, 31
];

gc = glitch.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset"])
#gc.set_range("width", 1, 8)
#gc.set_range("offset", -7, 3)

#gc.set_range("width", 5, 6)
#gc.set_range("offset", -2.5, -1)

gc.set_range("width", 5.5, 5.5)
gc.set_range("offset", -1.5, -1.5)
#gc.set_range("ext_offset", 1, scope.adc.trig_count)
gc.set_range("ext_offset", 1, scope.adc.trig_count)

gc.display_stats()

scope.clock.adc_src = "clkgen_x1"
scope.glitch.clk_src = "clkgen" # set glitch input clock
scope.glitch.output = "clock_xor" # glitch_out = clk ^ glitch
scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called
scope.io.hs2 = "glitch"  # output glitch_out on the clock line
scope.adc.timeout = 0.1

gc.set_global_step(1)
scope.glitch.repeat = 1
reboot_flush()

RUNS = 10000

from datetime import datetime
datetime_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

print(datetime_str)

for glitch_settings in gc.glitch_values():
    scope.glitch.width = glitch_settings[0]
    scope.glitch.offset = glitch_settings[1]
    scope.glitch.ext_offset = glitch_settings[2]
        
    data2 = np.zeros((RUNS,16),dtype=np.uint8)
    
    reset_cnt = 0
    valid_run = 1

    for i in range(RUNS):
        key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
        #                          SHARE1     SHARE2     RANDOMNESS
        #text = bytearray.fromhex("0000000000 0000000000 0000000000 00")
        #key  = bytearray.fromhex("0000000000 0000000000 0000000000 00")
        
        if scope.adc.state:
            gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
            reboot_flush()
            reset_cnt += 1
            if reset_cnt >= 10:
                valid_run = 0
                break
            continue
        
        try:
            trace = cw.capture_trace(scope, target, text, key)
        except:
            gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
            reboot_flush()
            reset_cnt += 1
            if reset_cnt >= 10:
                valid_run = 0
                break
            continue

        if trace is None:
            gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
            reboot_flush()
            reset_cnt += 1
            if reset_cnt >= 10:
                valid_run = 0
                break
            continue

        ciphertext = trace.textout
        if ciphertext is None:
            gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
            reboot_flush()
            reset_cnt += 1
            if reset_cnt >= 10:
                valid_run = 0
                break
            continue
            
        sbox_in = text
        sbox_out = ciphertext
        
        # input
        a = (sbox_in[0] ^ sbox_in[5]) & 0xFF
        b = (sbox_in[1] ^ sbox_in[6]) & 0xFF
        c = (sbox_in[2] ^ sbox_in[7]) & 0xFF
        d = (sbox_in[3] ^ sbox_in[8]) & 0xFF
        e = (sbox_in[4] ^ sbox_in[9]) & 0xFF
        
        # observed output
        aa = (sbox_out[0] ^ sbox_out[5]) & 0xFF
        bb = (sbox_out[1] ^ sbox_out[6]) & 0xFF
        cc = (sbox_out[2] ^ sbox_out[7]) & 0xFF
        dd = (sbox_out[3] ^ sbox_out[8]) & 0xFF
        ee = (sbox_out[4] ^ sbox_out[9]) & 0xFF
        
        # redundant computation
        aa_cor = a ^ ((b^0xFF) & c);
        bb_cor = b ^ ((c^0xFF) & d);
        cc_cor = c ^ ((d^0xFF) & e);
        dd_cor = d ^ ((e^0xFF) & a);
        ee_cor = e ^ ((a^0xFF) & b);
        
        ##assert(aa == aa_cor)
        ##assert(bb == bb_cor)
        ##assert(cc == cc_cor)
        ##assert(dd == dd_cor)
        ##assert(ee == ee_cor)
        
        data2[i,:] = [a,b,c,d,e,aa,bb,cc,dd,ee,aa_cor,bb_cor,cc_cor,dd_cor,ee_cor,1]

        if (aa!=aa_cor) or (bb!=bb_cor) or (cc!=cc_cor) or (dd!=dd_cor) or (ee!=ee_cor):
            gc.add("success", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
        else:
            gc.add("normal", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))

    if valid_run == 0:
        data2 = np.zeros((RUNS,16),dtype=np.uint8)

    with open('sefa_dom_' + datetime_str + '_' + str(scope.glitch.ext_offset) + '.npy' , 'wb') as f:
        np.save(f, data2)

In [ ]:
%matplotlib notebook
print(scope.glitch.ext_offset)
gc.results.plot_2d(plotdots={"success":"+g", "reset":"xr", "normal":None})

In [ ]:
%matplotlib notebook
print(scope.glitch.ext_offset)
gc.results.plot_2d(plotdots={"success":"+g", "reset":"xr", "normal":None})

In [ ]:
%matplotlib notebook
print(scope.glitch.ext_offset)
gc.results.plot_2d(plotdots={"success":"+g", "reset":"xr", "normal":None})

In [ ]:
print(scope.adc.trig_count)

In [ ]:
print(datetime_str)